In [ ]:
pip install symspellpy

In [ ]:
pip install contractions

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import pandas as pd
import unicodedata
import re
import numpy as np
import os
import io
import time
import json
import math

# Spelling Correction Library
from symspellpy import SymSpell 

# Library to create a dictionary with word and its freq in the given text
from collections import Counter

# Library to handle expand contracted words based on the context.
import contractions



In [ ]:
df = pd.read_json('/content/drive/MyDrive/text2sql-main/text2sql-main/spider/spider/train_spider.json')

In [ ]:
df

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."
...,...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,"[SELECT, T1.company_name, FROM, culture_compan...","[select, t1, ., company_name, from, culture_co...",What are all the company names that have a boo...,"[What, are, all, the, company, names, that, ha...","{'from': {'table_units': [['table_unit', 2], [..."
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",Show the movie titles and book titles for all ...,"[Show, the, movie, titles, and, book, titles, ...","{'from': {'table_units': [['table_unit', 1], [..."
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",What are the titles of movies and books corres...,"[What, are, the, titles, of, movies, and, book...","{'from': {'table_units': [['table_unit', 1], [..."
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,"[SELECT, T2.company_name, FROM, movie, AS, T1,...","[select, t2, ., company_name, from, movie, as,...",Show all company names with a movie directed i...,"[Show, all, company, names, with, a, movie, di...","{'from': {'table_units': [['table_unit', 1], [..."


In [ ]:
df.columns

Index(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question',
       'question_toks', 'sql'],
      dtype='object')

In [ ]:
df.db_id.nunique()

140

In [ ]:
df.db_id.unique()

array(['department_management', 'farm', 'student_assessment', 'bike_1',
       'book_2', 'musical', 'twitter_1', 'product_catalog', 'flight_1',
       'allergy_1', 'store_1', 'journal_committee',
       'customers_card_transactions', 'race_track', 'coffee_shop',
       'chinook_1', 'insurance_fnol', 'medicine_enzyme_interaction',
       'university_basketball', 'phone_1', 'match_season', 'climbing',
       'body_builder', 'election_representative', 'apartment_rentals',
       'game_injury', 'soccer_1', 'performance_attendance', 'college_2',
       'debate', 'insurance_and_eClaims', 'customers_and_invoices',
       'wedding', 'theme_gallery', 'epinions_1', 'riding_club', 'gymnast',
       'small_bank_1', 'browser_web', 'wrestler', 'school_finance',
       'protein_institute', 'cinema', 'products_for_hire', 'phone_market',
       'gas_company', 'party_people', 'pilot_record',
       'cre_Doc_Control_Systems', 'company_1', 'local_govt_in_alabama',
       'formula_1', 'machine_repair', 'en

In [ ]:
def data_import(file='train.json', max_length=200):
  with open(file, 'r') as f:
      split_data = json.load(f)

  query = []
  utterance = []

  for i in split_data:
      q = i['query']
      u = i['question']
      query.append(q)
      utterance.append(u)  
  
  return query, utterance


In [ ]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  w = re.sub(r"([^A-z0-9 ])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = w.strip()

  # adding contractions
  w = contractions.fix(w)
    
  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.

  w = '<start> ' + w + ' <end>'
  return w


def create_dataset(path, num_examples, show_example=False):
  lines = path
  word_pairs = []
  for l in lines:
    word =preprocess_sentence(l)
    word_pairs.append(word)
  
  if show_example:
    print(word_pairs[0])
  return  word_pairs

In [ ]:
def creating_vocab(data_list):
  corpus = []
  for data in data_list:
    for sentence in data:
      tokens = sentence.split()
      corpus.extend(tokens)
  
  corpus = Counter(corpus)
  with open('dictionary.txt', 'w') as f:
    for k, v in corpus.items():
      f.write(f"{k} {v}\n")
    
def spell_checker(inputTerm, path = './dictionary.txt'):
  symspell = SymSpell()
  symspell.load_dictionary(path, term_index=0, count_index=1)
  maxEditDistance = 2
  # ignore_non_words = True means if a particular word is not present then
  # we'll return as is.
  correct_sent = []
  for i in inputTerm.split():
    if i.isalnum():
      suggestion = symspell.lookup_compound(i, maxEditDistance, ignore_non_words=True)
      suggestion = str(suggestion[0]).split(',')[0].strip()
    else:
      suggestion = i.strip()  
    correct_sent.append(suggestion)
  
  return " ".join(correct_sent)

In [ ]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='',oov_token="<unk>")
  lang_tokenizer.fit_on_texts(lang)
  
  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

def glove_embeddings(GLOVE_DIR, GLOVE_FILE, EMBEDDING_DIM):
  embeddings_index = {}
  f = open(os.path.join(GLOVE_DIR, GLOVE_FILE))
  emb_matrix = []
  for line in f:
      values = line.split()
      word = " ".join(values[:-1*EMBEDDING_DIM])
      coefs = np.asarray(values[-1*EMBEDDING_DIM:], dtype='float32')      
      embeddings_index[word] = coefs
      emb_matrix.append(coefs)
  f.close()

  embeddings_index['<unk>'] = np.mean(emb_matrix, axis=0) 
  
  scale = 1/max(1., (len(emb_matrix[0]))/2.)
  limit = math.sqrt(3.0 * scale)
  embeddings_index['<start>'] = np.random.uniform(-limit, limit, size=(1,EMBEDDING_DIM))
  embeddings_index['<end>'] = np.random.uniform(-limit, limit, size=(1,EMBEDDING_DIM))

  print('Found %s word vectors.' % len(embeddings_index))

  return embeddings_index

def embedding_matrix(embeddings_index, EMBEDDING_DIM, word_index):
  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
      else:
        embedding_matrix[i] = embeddings_index['<unk>'] 
        + np.random.rand(1,EMBEDDING_DIM)
  return embedding_matrix

In [ ]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [ ]:
def store_embedding(emb, filename='Embedding'):
  emb.tofile(f'{filename}.dat')
  with open(f'{filename}_shape_info.txt', 'w') as f:
    f.write(f"{str(emb.shape[0])} {str(emb.shape[1])}")

def retrieve_embedding(filename = 'Embedding'):
  with open(f'{filename}_shape_info.txt', 'r') as f:
    for i in f:
      shape = (int(i.split()[0]), int(i.split()[1]))

  emb_matrix = np.fromfile(f'{filename}.dat').reshape(shape)
  return emb_matrix


In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz, emb_type='default', emb_mat=[]):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    if emb_type == 'glove':
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[emb_mat])
    else:
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
      
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
class LuongAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(LuongAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    query_with_time_axis = tf.expand_dims(query, 1)


    values_transposed = tf.transpose(values, perm=[0, 2, 1])

    #BAHDANAU ADDITIVE:
    #score = self.V(tf.nn.tanh(
    #    self.W1(query_with_time_axis) + self.W2(values)))
    
    #LUONG Dot-product
    score = tf.transpose(tf.matmul(query_with_time_axis, values_transposed) , perm=[0, 2, 1])

    attention_weights = tf.nn.softmax(score, axis=1)

    context_vector = attention_weights * values

    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, emb_type='default', emb_mat=[]):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units

    if emb_type == 'glove':
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[emb_mat])
    else:
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
      
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = LuongAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)

    x = self.embedding(x)
 
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
# Setting OS Path to data folder
os.chdir('/content/drive/MyDrive/text2sql-main/text2sql-main/data/')

# Data Import
query, utterance = data_import('train_spider.json')

# Variables
embedding_dim = 50
glove_file_name = 'glove.6B.50d.txt'
# We can use either glove.6B.50d.txt or glove.840B.300d.txt

# Calling data cleaning and processing function
print('---Data after preprocessing and cleaning---')
en = create_dataset(utterance, None, show_example=True)
sql = create_dataset(query, None, show_example=True)

# Creating a spell checker based on given data to correct the input at 
# inference time
creating_vocab([en, sql])
example_sent = 'what is the number ofeemployees in each department ?'
print(f'\nSpell Checker:\nBefore: {example_sent}')
print(f'After: {spell_checker(example_sent)} \n')

# Tokenization
targ_lang, inp_lang = sql, en
input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

# Extracting GloVe Embeddings
print('\n---Extracting Embedding Matrix---')
embeddings_index = glove_embeddings('./', glove_file_name, embedding_dim)

# Embedding Matrix
input_emb_matrix = embedding_matrix(embeddings_index, embedding_dim, inp_lang_tokenizer.word_index)
target_emb_matrix = embedding_matrix(embeddings_index, embedding_dim, targ_lang_tokenizer.word_index)

#storing embedding matrix
store_embedding(input_emb_matrix, 'input_emb_matrix')
store_embedding(target_emb_matrix, 'target_emb_matrix')

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print('\n---Data after Train-Test Split---')
print('Train-Input, Train-Target, Val-Input, Val-Target')
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

inp_lang = inp_lang_tokenizer
targ_lang = targ_lang_tokenizer

# Showing Mapping between words and indices
print ("\nInput Language; index to word mapping")
convert(inp_lang_tokenizer, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang_tokenizer, target_tensor_train[0])

---Data after preprocessing and cleaning---
<start> how many heads of the departments are older than 56 ? <end>
<start> select count ( * ) from head where age > 56 <end>

Spell Checker:
Before: what is the number ofeemployees in each department ?
After: what is the number of employees in each department ? 


---Extracting Embedding Matrix---
Found 400003 word vectors.

---Data after Train-Test Split---
Train-Input, Train-Target, Val-Input, Val-Target
5600 5600 1400 1400

Input Language; index to word mapping
3 ----> <start>
8 ----> what
10 ----> are
2 ----> the
810 ----> draft
1909 ----> pick
165 ----> numbers
9 ----> and
810 ----> draft
320 ----> classes
21 ----> for
81 ----> players
23 ----> who
345 ----> play
2 ----> the
1138 ----> defender
183 ----> position
6 ----> ?
4 ----> <end>

Target Language; index to word mapping
8 ----> <start>
5 ----> select
1955 ----> draft_pick_number
17 ----> ,
1956 ----> draft_class
6 ----> from
218 ----> match_season
13 ----> where
153 ----> position

In [ ]:
print(en)

['<start> how many heads of the departments are older than 56 ? <end>', '<start> list the name , born state and age of the heads of departments ordered by age . <end>', '<start> list the creation year , name and budget of each department . <end>', '<start> what are the maximum and minimum budget of the departments ? <end>', '<start> what is the average number of employees of the departments whose rank is between 10 and 15 ? <end>', '<start> what are the names of the heads who are born outside the california state ? <end>', "<start> what are the distinct creation years of the departments managed by a secretary born in state ' alabama ' ? <end>", '<start> what are the names of the states where at least 3 heads were born ? <end>', '<start> in which year were most departments established ? <end>', "<start> show the name and number of employees for the departments managed by heads whose temporary acting value is ' yes ' ? <end>", '<start> how many acting statuses are there ? <end>', '<start

In [ ]:
print(input_tensor)

[[ 3 24 26 ...  0  0  0]
 [ 3 29  2 ...  0  0  0]
 [ 3 29  2 ...  0  0  0]
 ...
 [ 3  8 10 ...  0  0  0]
 [ 3 25 15 ...  0  0  0]
 [ 3  8 10 ...  0  0  0]]


In [ ]:
print(inp_lang_tokenizer)

In [ ]:
print(input_emb_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.12920076 -0.28866628 -0.01224866 ...  0.10069408  0.00653003
   0.01685157]
 [ 0.41800001  0.24968    -0.41242    ... -0.18411    -0.11514
  -0.78580999]
 ...
 [ 0.66749001 -0.41321     0.065755   ... -0.96570998 -0.26875001
  -1.17410004]
 [ 0.021355   -0.13042     0.028404   ...  1.301      -0.072261
   0.075404  ]
 [-0.0080708  -0.15383001 -0.72074997 ... -0.81581998 -0.15831
  -0.085936  ]]


In [ ]:
input_emb_matrix.shape

(3062, 50)

In [ ]:
# Defining Variables for Modelling
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 32
embedding_dim = 50
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
steps_per_epoch_val = len(input_tensor_val)//BATCH_SIZE
units = 1024
vocab_inp_size = len(inp_lang_tokenizer.word_index)+1
vocab_tar_size = len(targ_lang_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset_val = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
print(example_input_batch.shape, example_target_batch.shape)

# Encoder
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, 'glove', input_emb_matrix)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

# Attention Layer
attention_layer = LuongAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

# Decoder
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 'glove', target_emb_matrix)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))



(32, 47) (32, 134)
Encoder output shape: (batch size, sequence length, units) (32, 47, 1024)
Encoder Hidden state shape: (batch size, units) (32, 1024)
Attention result shape: (batch size, units) (32, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (32, 47, 1)
Decoder output shape: (batch_size, vocab size) (32, 2820)


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/text2sql-main/text2sql-main/ckpt'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_with_luong")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1])) 

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

def val_step(inp, targ, enc_hidden):
  loss = 0

  enc_output, enc_hidden = encoder(inp, enc_hidden)

  dec_hidden = enc_hidden

  dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

  # Teacher forcing - feeding the target as the next input
  for t in range(1, targ.shape[1]):
    # passing enc_output to the decoder
    predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

    loss += loss_function(targ[:, t], predictions)

    # using teacher forcing
    dec_input = tf.expand_dims(targ[:, t], 1) 

  batch_loss = (loss / int(targ.shape[1])) 


  return batch_loss

In [ ]:
EPOCHS = 100
epoch_train_loss = []
epoch_val_loss = []


for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  total_loss_val = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

  for (batch, (inp, targ)) in enumerate(dataset_val.take(steps_per_epoch_val)):
    batch_loss_val = val_step(inp, targ, enc_hidden)
    total_loss_val += batch_loss_val

  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Train Loss {:.4f} Val Loss {:.4f}'.
        format(
            epoch + 1,
            total_loss / steps_per_epoch,
            total_loss_val / steps_per_epoch_val))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

  epoch_train_loss.append(total_loss / steps_per_epoch)
  epoch_val_loss.append(total_loss_val / steps_per_epoch_val)

Epoch 1 Train Loss 0.6327 Val Loss 0.4667
Time taken for 1 epoch 254.62992978096008 sec

Epoch 2 Train Loss 0.4077 Val Loss 0.3595
Time taken for 1 epoch 104.86541366577148 sec

Epoch 3 Train Loss 0.3300 Val Loss 0.3037
Time taken for 1 epoch 104.27845573425293 sec

Epoch 4 Train Loss 0.2669 Val Loss 0.2566
Time taken for 1 epoch 104.63382005691528 sec

Epoch 5 Train Loss 0.2172 Val Loss 0.2243
Time taken for 1 epoch 104.15962743759155 sec

Epoch 6 Train Loss 0.1818 Val Loss 0.1967
Time taken for 1 epoch 104.44656467437744 sec

Epoch 7 Train Loss 0.1504 Val Loss 0.1803
Time taken for 1 epoch 104.05455565452576 sec

Epoch 8 Train Loss 0.1240 Val Loss 0.1681
Time taken for 1 epoch 104.12719750404358 sec

Epoch 9 Train Loss 0.1047 Val Loss 0.1611
Time taken for 1 epoch 103.57552528381348 sec

Epoch 10 Train Loss 0.0955 Val Loss 0.1559
Time taken for 1 epoch 103.86265349388123 sec

Epoch 11 Train Loss 0.0902 Val Loss 0.1632
Time taken for 1 epoch 103.31006288528442 sec

Epoch 12 Train Loss

In [ ]:
## restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
def evaluate(sentence, preprocess=False):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  if preprocess:
    sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = []

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    predicted_id = tf.argmax(predictions[0]).numpy()
    if targ_lang.index_word[predicted_id] == '<end>':
      return " ".join(result), sentence, attention_plot    

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result.append(targ_lang.index_word[predicted_id])

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return " ".join(result), sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1) 
  ax.matshow(attention, cmap='viridis')
  for (i, j), z in np.ndenumerate(attention):
    ax.text(j, i, '{:0.1f}'.format(z), ha='center', va='center')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()


In [ ]:
def translate(sentence, plot=True, preprocess=False):
  result, sentence, attention_plot = evaluate(sentence, preprocess)

  if plot==True:
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))

  return result


In [ ]:
def jaccard_similarity(candidate, reference):
    """Returns the Jaccard similarity between two token lists

    Args:
        candidate (list of int): tokenized version of the candidate translation
        reference (list of int): tokenized version of the reference translation

    Returns:
        float: overlap between the two token lists
    """
    
    # convert the lists to a set to get the unique tokens
    can_unigram_set, ref_unigram_set = set(candidate), set(reference)  
    
    # get the set of tokens common to both candidate and reference
    joint_elems = can_unigram_set.intersection(ref_unigram_set)
    
    # get the set of all tokens found in either candidate or reference
    all_elems = can_unigram_set.union(ref_unigram_set)
    
    # divide the number of joint elements by the number of all elements
    overlap = len(joint_elems) / len(all_elems)
    
    return overlap

In [ ]:
# for making a frequency table easily
from collections import Counter

def rouge1_similarity(system, reference):
    """Returns the ROUGE-1 score between two token lists

    Args:
        system (list of int): tokenized version of the system translation
        reference (list of int): tokenized version of the reference translation

    Returns:
        float: overlap between the two token lists
    """        
    precision, recall, rougel_score = 0, 0, 0
    # make a frequency table of the system tokens
    sys_counter = Counter(system)
    
    # make a frequency table of the reference tokens
    ref_counter = Counter(reference)
    
    # initialize overlap to 0
    overlap = 0
    
    # run a for loop over the sys_counter object
    for token in sys_counter:
        
        # lookup the value of the token in the sys_counter dictionary
        token_count_sys = sys_counter.get(token,0)
        
        # lookup the value of the token in the ref_counter dictionary
        token_count_ref = ref_counter.get(token,0)
        
        # update the overlap by getting the smaller number between the two token counts above
        overlap += min(token_count_sys, token_count_ref)
    
    precision = overlap / sum(sys_counter.values())
    recall = overlap / sum(ref_counter.values())
    
    if precision + recall != 0:
        # compute the f1-score
        rouge1_score = 2 * ((precision * recall)/(precision + recall))
    else:
        rouge1_score = 0 

    return precision, recall, rouge1_score

In [ ]:
def prediction(utterance_data, query_data):
  n = len(utterance_data)
  pred = []
  targets = []
  for i in range(n):
    actual = []
    target = []
    for j in utterance_data[i]:
      if j not in [0]:
        actual.append(inp_lang_tokenizer.index_word[j])
    
    actual = " ".join(actual)

    for j in query_data[i]:
      if j not in [0]:
        if targ_lang_tokenizer.index_word[j] not in ['<start>', '<end>']:
          target.append(targ_lang_tokenizer.index_word[j])
    
    targets.append(" ".join(target))  
    pred.append(translate(f"{actual}", plot=False))
  
  return pred, targets

def evaluation_metric(pred, target, method='jaccard'):
  scores = []
  precisions = []
  recalls = []
  n = len(pred)
  for i in range(n):
    if method == 'jaccard':
      score = jaccard_similarity(pred[i].split(), target[i].split())
    elif method == 'rougel':
      precision, recall, score = rouge1_similarity(pred[i].split(), target[i].split())
      precisions.append(precision)
      recalls.append(recall)
    elif method == 'bleu':
      score = bleu_score(pred[i].split(), target[i].split())
  
    scores.append(score)
  
  if method == 'rougel':
    return np.mean(scores), np.mean(precisions), np.mean(recalls)
  else:
    return np.mean(scores)

In [ ]:
pred_train, targets_train = prediction(input_tensor_train, target_tensor_train)
pred_val, targets_val = prediction(input_tensor_val, target_tensor_val)

In [ ]:
jaccard_score_train = evaluation_metric(pred_train, targets_train)

In [ ]:
precision_train, recall_train, rougel_score_train = evaluation_metric(pred_train, targets_train, method='rougel')

In [ ]:
jaccard_score_val = evaluation_metric(pred_val, targets_val)

In [ ]:
precision_val, recall_val, rougel_score_val = evaluation_metric(pred_val, targets_val, method='rougel')

In [ ]:
F1_train = (2*(precision_train * recall_train))/(precision_train+recall_train)
F1_train

0.8071171817591931

In [ ]:
F1_val = (2*(precision_val * recall_val))/(precision_val+recall_val)
F1_val

0.8015432639513197

In [ ]:
def metric_print(precision, recall,F1_score, rouge1_score, jaccard_similarity, text):
  print(f'---{text}---')
  print(f'Precision: {np.round(precision, 4)}')
  print(f'Recall: {np.round(recall, 4)}')
  print(f'F1_score:{np.round(F1_score,4)}')
  print(f'Rouge1 Similarity Score: {np.round(rouge1_score, 4)}')
  print(f'Jaccard Similarity: {np.round(jaccard_similarity, 4)}')
  print('\n')

In [ ]:
metric_print(precision_train, recall_train,F1_train, rougel_score_train, jaccard_score_train, 'Evaluation Metrics on Training Dataset: ')
metric_print(precision_val, recall_val,F1_val, rougel_score_val, jaccard_score_val, 'Evaluation Metrics on Validation Dataset: ')

---Evaluation Metrics on Training Dataset: ---
Precision: 0.7938
Recall: 0.8209
F1_score:0.8071
Rouge1 Similarity Score: 0.8564
Jaccard Similarity: 0.8532


---Evaluation Metrics on Validation Dataset: ---
Precision: 0.7889
Recall: 0.8146
F1_score:0.8015
Rouge1 Similarity Score: 0.8559
Jaccard Similarity: 0.8532




In [ ]:
evaluate("How many students are there?", preprocess=True)

('select count ( * ) from student',
 '<start> how many students are there ? <end>',
 array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.27207518e-01, 1.45093888e-01, 1.64012000e-01],
        [2.86311667e-25, 4.89421816e-26, 5.33866509e-27, ...,
         2.80676676e-16, 2.72277867e-16, 2.64564018e-16],
        [2.12141519e-16, 1.55406806e-17, 1.72469926e-15, ...,
         1.42976370e-18, 1.30560916e-18, 1.19625392e-18],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]))

In [ ]:
translate("How many students are there?", preprocess=True ,plot=False)

'select count ( * ) from student'

In [ ]:
translate("How many female students are there?", preprocess=True ,plot=False)

"select count ( * ) from student where sex = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and allergy = ' f ' and"

In [ ]:
translate('What are the ids of the students who registered for course 301?',preprocess=True,plot=False)

'select student_id from student_course_attendance where course_id = 301'

In [ ]:
translate("What is the name of a student with minimum age?", preprocess=True,plot=False)

"select name from student where job = ' history ' )"

In [ ]:
translate("How many employees are there in each and every department?", preprocess=True,plot=False)

'select count ( * ) from employees'

In [ ]:
translate("Tell me the name of all students?", preprocess=True, plot=False)

'select name from student'

In [ ]:
translate("What is the minimum age of a student?", preprocess=True, plot=False)

'select min ( age ) from student group by sex'

In [ ]:
translate("How many acting statuses are there?", preprocess=True, plot=False)

'select count ( distinct temporary_acting ) from management'

In [ ]:
translate("For each zip code, return how many times max wind speed reached 25?", preprocess=True, plot=False)

'select zip_code , count ( * ) from weather where max_wind_speed_mph > = 25'

In [ ]:
model_output = translate("How many acting statuses are there?", preprocess=True,plot=False)
model_op = model_output
model_op

'select count ( distinct temporary_acting ) from management'

In [ ]:
from sqlalchemy import create_engine

In [ ]:
my_conn = create_engine("sqlite:////content/drive/MyDrive/text2sql-main/text2sql-main/spider/spider/database/department_management/department_management.sqlite")

In [ ]:
r_set = my_conn.execute(model_op)
for row in r_set:
  print(row)

(2,)


In [ ]:
translate('Find the name of all students who were in the tryout sorted in alphabetic order.',preprocess=True,plot=False)

'select t1 . pname from player as t1 join tryout as t2 on t1 . pid = t2 . pid order by t1 . pname from player as t1 join tryout as t2 on t1 . pid = t2 . pid order by t1 . pname from player as t1 join tryout as t2 on t1 . pid = t2 . pid order by t1 . pname from player as t1 join tryout as t2 on t1 . pid = t2 . pid order by t1 . pname from player as t1 join tryout as t2 on t1 . pid = t2 . pid order by t1 . pname from player as t1 join tryout as t2 on t1 . pid = t2 . pid order by t1 . pname from player as t1'

In [ ]:
model_output = translate("For each zip code, return how many times max wind speed reached 25?", preprocess=True,plot=False)
model_op1 = model_output
model_op1

'select zip_code , count ( * ) from weather where max_wind_speed_mph > = 25'

In [ ]:
my_conn = create_engine("sqlite:////content/drive/MyDrive/text2sql-main/text2sql-main/spider/spider/database/bike_1/bike_1.sqlite")

In [ ]:
r_set = my_conn.execute(model_op1)
for row in r_set:
  print(row)

OperationalError: ignored

In [ ]:
r_set = my_conn.execute(''' select zip_code , count ( * ) from weather where max_wind_speed_mph >= 25 ''')
for row in r_set:
  print(row)

(95113, 171)


In [ ]:
model_output = translate("How many students are there?", preprocess=True,plot=False)
model_op2 = model_output
model_op2

'select count ( * ) from student'

In [ ]:
my_conn = create_engine("sqlite:////content/drive/MyDrive/text2sql-main/text2sql-main/spider/spider/database/dorm_1/dorm_1.sqlite")

In [ ]:
r_set = my_conn.execute(model_op2)
for row in r_set:
  print(row)

(34,)


In [ ]:
df['table_name'] = [x.split('FROM')[1].split(' ')[1] for x in df['query']]

In [ ]:
df

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,table_name
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ...",head
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ...",head
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ...",department
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ...",department
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ...",department
...,...,...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,"[SELECT, T1.company_name, FROM, culture_compan...","[select, t1, ., company_name, from, culture_co...",What are all the company names that have a boo...,"[What, are, all, the, company, names, that, ha...","{'from': {'table_units': [['table_unit', 2], [...",culture_company
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",Show the movie titles and book titles for all ...,"[Show, the, movie, titles, and, book, titles, ...","{'from': {'table_units': [['table_unit', 1], [...",movie
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...",What are the titles of movies and books corres...,"[What, are, the, titles, of, movies, and, book...","{'from': {'table_units': [['table_unit', 1], [...",movie
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,"[SELECT, T2.company_name, FROM, movie, AS, T1,...","[select, t2, ., company_name, from, movie, as,...",Show all company names with a movie directed i...,"[Show, all, company, names, with, a, movie, di...","{'from': {'table_units': [['table_unit', 1], [...",movie


In [ ]:
df_table_mapping = df.set_index('table_name')['db_id'].to_dict()

In [ ]:
df_table_mapping

{'': 'tracking_share_transactions',
 '(': 'baseball_1',
 '(SELECT': 'flight_4',
 'ADDRESSES': 'behavior_monitoring',
 'ALBUM': 'chinook_1',
 'APPELLATIONS': 'wine_1',
 'APPELLATIONs': 'wine_1',
 'ARTIST': 'chinook_1',
 'ASSESSMENT_NOTES': 'behavior_monitoring',
 'Accounts': 'cre_Docs_and_Epenses',
 'Activity': 'activity_1',
 'Addresses': 'driving_school',
 'Aircraft': 'flight_1',
 'Album': 'chinook_1',
 'Albums': 'music_2',
 'All_documents': 'cre_Doc_Tracking_DB',
 'Allergy_type': 'allergy_1',
 'Apartment_Bookings': 'apartment_rentals',
 'Apartment_Buildings': 'apartment_rentals',
 'Apartment_Facilities': 'apartment_rentals',
 'Apartments': 'apartment_rentals',
 'Artwork': 'entertainment_awards',
 'Assessment_Notes': 'behavior_monitoring',
 'Assets': 'assets_maintenance',
 'Attribute_Definitions': 'product_catalog',
 'BLOCK': 'hospital_1',
 'BOOKINGS': 'cre_Drama_Workshop_Groups',
 'Band': 'music_2',
 'Behavior_Incident': 'behavior_monitoring',
 'Bookings': 'cre_Drama_Workshop_Groups',